In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\kidney-disease-classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Projects\\kidney-disease-classification'

In [5]:
import dagshub
dagshub.init(repo_owner='28p07', repo_name='Kidney-disease-classification', mlflow=True)

Accessing as 28p07

Initialized MLflow to track repo "28p07/Kidney-disease-classification"

Repository 28p07/Kidney-disease-classification initialized!

In [6]:
os.environ['MLFLOW_TRACKING_USERNAME'] = '28p07'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '9e22b5bf0a0094e4bc3b1e1e21e40123d5026f2d'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/28p07/Kidney-disease-classification.mlflow'

In [7]:
import tensorflow as tf

In [8]:
##entity
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [10]:
## configuration.py

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model = 'artifacts/training/model.h5',
                training_path = 'artifacts\data_ingestion\kidney_disease_img_dataset',
                all_params = self.params,
                mlflow_uri = 'https://dagshub.com/28p07/Kidney-disease-classification.mlflow',
                params_image_size = self.params.IMAGE_SIZE,
                params_batch_size = self.params.BATCH_SIZE 
          )

          return eval_config



In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.set_experiment('vgg')

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, artifact_path = "model", registered_model_name="VGG16")
            else:
               mlflow.keras.log_model(self.model, "model")


In [13]:
## pipeline

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-09-29 13:06:45,017: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-29 13:06:45,022: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-29 13:06:45,025: INFO: common: directory created at :{path}]
Found 300 images belonging to 2 classes.
19/19 [==============================] - 47s 2s/step - loss: 25.4917 - accuracy: 0.5000
[2024-09-29 13:07:33,003: INFO: common: json file saved at scores.json]


2024/09/29 13:07:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-09-29 13:07:43,089: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\Acer\AppData\Local\Temp\tmp5sbj8j51\model\data\model\assets
[2024-09-29 13:07:44,024: INFO: builder_impl: Assets written to: C:\Users\Acer\AppData\Local\Temp\tmp5sbj8j51\model\data\model\assets]


2024/09/29 13:08:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16' already exists. Creating a new version of this model...
2024/09/29 13:08:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16, version 5
Created version '5' of model 'VGG16'.
2024/09/29 13:08:37 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-foal-852 at: https://dagshub.com/28p07/Kidney-disease-classification.mlflow/#/experiments/2/runs/f4e28c8d218f464ea31759508d5b27c3.
2024/09/29 13:08:37 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/28p07/Kidney-disease-classification.mlflow/#/experiments/2.
